In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Complicaciones Quirúrgicas

En este cuadernillo se quieren identificar y cuantificar las complicaciones quirúrgicas en el INT.

In [ ]:
RUTA_PABELLON = "../data/external/datos_limpios_pabellon.csv"

# Leer y formatear datos de operaciones quirúrgicas
df_pabellon = pd.read_csv(RUTA_PABELLON, encoding="latin-1", sep=";")
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

# Limpiar datos de operaciones quirúrgicas
df_pabellon_limpio = build_features.limpiar_datos_operaciones_quirurgicas(df_pabellon)
df_pabellon_limpio = df_pabellon_limpio.reset_index(names="id_int_q")

In [ ]:
def buscar_nombre_procedimiento_pabellon(df, operaciones):
    df_de_operacion = df[
        df["nombre_de_la_operacion"].fillna("").str.contains(operaciones, regex=True)
    ]

    resumen_operacion = pd.DataFrame(
        {
            "totales": df.groupby(df.fecha.dt.year).size(),
            "operacion": df_de_operacion.groupby(df_de_operacion.fecha.dt.year).size(),
        }
    )
    resumen_operacion["porcentaje"] = (
        resumen_operacion["operacion"] / resumen_operacion["totales"]
    ) * 100

    return df_de_operacion, resumen_operacion

In [ ]:
a_buscar_operaciones = {
    "ecmo": r"ECMO",
    "trasplantes": r"TRASPLANTE|TRANSPLANTE",
    "aseos": r"ASEO",
    "drenajes": r"DRENAJE",
    "traqueostomias": r"TRAQUEOSTOMIA|TRAQUEOSTOMÍA",
    "reintervenciones_reoperaciones": r"REINTERVENCION|REOPERACION|REINTERVENCIÓN|REOPERACIÓN",
}

a_buscar_diagnosticos = {
    "empiemas": r"EMPIEMA"
}

In [ ]:
resultados_operaciones = {}
for tipo_complicacion, textos_a_buscar in a_buscar_operaciones.items():
    busqueda = buscar_nombre_procedimiento_pabellon(df_pabellon_limpio, textos_a_buscar)
    resultados_operaciones[tipo_complicacion] = busqueda